In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import joblib

# Load the dataset
file_path = "vibration_data_with_OEE.xlsx"
df = pd.read_excel(file_path, sheet_name="vibration_data_with_OEE")

# Drop timestamp (not useful for model training)
df = df.drop(columns=["Timestamp"], errors='ignore')

# Ensure OEE is calculated if not available
df["OEE"] = df["Availability"] * df["Performance"] * df["Quality"]

# Define input features (X) and target variable (y)
X = df.drop(columns=["OEE"])
y = df["OEE"]

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize features for better model performance
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train a Random Forest Regressor model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

# Predictions
y_pred = model.predict(X_test_scaled)

# Evaluate model performance
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Model Performance:\nMean Absolute Error: {mae}\nR² Score: {r2}")

# Save the model and scaler for live data predictions
joblib.dump(model, "oee_model.pkl")
joblib.dump(scaler, "scaler.pkl")
print("Model and scaler saved successfully.")


FileNotFoundError: [Errno 2] No such file or directory: 'vibration_data_with_OEE.xlsx'

In [6]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import joblib

# ✅ Load dataset
DATA_FILE = "vibration_data_with_OEE.csv"
df = pd.read_csv(DATA_FILE, encoding="latin1")

# ✅ Select only the 6 available features
FEATURES = [
    "Z-Axis RMS Velocity (in/sec)", 
    "X-Axis RMS Velocity (in/sec)",
    "Temperature (°F)",
    "Z-Axis Peak Acceleration (G)",
    "X-Axis Peak Acceleration (G)",
    "Z-Axis Peak Velocity (mm/sec)",
]


target = "OEE"

# ✅ Drop missing values
df = df.dropna()

# ✅ Split data into training and testing sets
X = df[FEATURES]
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ✅ Scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ✅ Train a new model using only the 6 features
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

# ✅ Save the new model and scaler
joblib.dump(model, "oee_model_tuned.pkl")
joblib.dump(scaler, "scaler3.pkl")

print("✅ Model retrained with 6 features and saved!")


✅ Model retrained with 6 features and saved!


In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import joblib
from pymodbus.client import ModbusSerialClient
import time
import csv

# ✅ Load Trained Model and Scaler
model = joblib.load("oee_model_tuned.pkl")
scaler = joblib.load("scaler3.pkl")

# ✅ Configure Modbus RTU Client
client = ModbusSerialClient(
    port="COM7",  # Change based on your system
    baudrate=19200,
    stopbits=1,
    bytesize=8,
    parity="N",
    timeout=3
)

# ✅ Register Addresses and Scaling Factors
REGISTERS = {  
    "Z-Axis RMS Velocity (in/sec)": 45201 - 40001,
    "Z-Axis RMS Velocity (mm/sec)": 45202 - 40001,
    "Temperature (°F)": 45203 - 40001,
    "Temperature (°C)": 45204 - 40001,
    "X-Axis RMS Velocity (in/sec)": 45205 - 40001,
    "X-Axis RMS Velocity (mm/sec)": 45206 - 40001,
}

SCALING_FACTORS = {
    "Z-Axis RMS Velocity (in/sec)": 6.5535 / 65535,
    "Z-Axis RMS Velocity (mm/sec)": 65.535 / 65535,
    "Temperature (°F)": 327.67 / 32767,
    "Temperature (°C)": 327.67 / 32767,
    "X-Axis RMS Velocity (in/sec)": 6.5535 / 65535,
    "X-Axis RMS Velocity (mm/sec)": 65.535 / 65535,
}

# ✅ Create CSV File
DATA_FILE = "live_oee_predictions.csv"
with open(DATA_FILE, mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["Timestamp"] + list(REGISTERS.keys()) + ["Predicted OEE"])

# ✅ Function to Read Modbus Register
def read_register(address, scale_factor):
    """Reads Modbus register and applies scaling factor."""
    try:
        response = client.read_holding_registers(address=address, count=1, slave=1)
        if response and response.registers:
            raw_value = response.registers[0]
            return round(raw_value * scale_factor, 3)
        else:
            return None
    except Exception as e:
        print(f"Error reading register {address}: {e}")
        return None

# ✅ Real-time Data Collection and OEE Prediction
def collect_data_and_predict_oee():
    if client.connect():
        print("✅ Connected to Modbus RTU device")
    else:
        print("❌ Failed to connect")
        return

    while True:
        timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
        data_row = [timestamp]
        features = []

        # Collect sensor data
        for param, reg in REGISTERS.items():
            value = read_register(reg, SCALING_FACTORS[param])
            data_row.append(value)
            features.append(value)

        # Convert live data to DataFrame with correct feature names
        features_df = pd.DataFrame([features], columns=REGISTERS.keys())

        # Scale the data using the saved scaler
        features_scaled = scaler.transform(features_df)

        # Predict OEE
        predicted_oee = model.predict(features_scaled)[0]
        data_row.append(round(predicted_oee, 3))

        # Save data to CSV
        with open(DATA_FILE, mode="a", newline="") as file:
            writer = csv.writer(file)
            writer.writerow(data_row)

        print(f"📊 Data Saved: {data_row}")
        time.sleep(1)  # Adjust as needed

# Start data collection and prediction
collect_data_and_predict_oee()
client.close()


✅ Connected to Modbus RTU device
📊 Data Saved: ['2025-02-20 10:59:04', 0.034, 0.876, 90.24, 32.35, 0.016, 0.412, 0.089]
📊 Data Saved: ['2025-02-20 10:59:05', 0.034, 0.876, 90.24, 32.35, 0.017, 0.422, 0.089]
📊 Data Saved: ['2025-02-20 10:59:06', 0.035, 0.879, 90.33, 32.4, 0.017, 0.422, 0.089]
📊 Data Saved: ['2025-02-20 10:59:07', 0.035, 0.879, 90.33, 32.4, 0.017, 0.422, 0.089]
📊 Data Saved: ['2025-02-20 10:59:08', 0.035, 0.879, 90.33, 32.4, 0.017, 0.422, 0.089]
📊 Data Saved: ['2025-02-20 10:59:10', 0.035, 0.879, 90.33, 32.4, 0.017, 0.422, 0.089]
📊 Data Saved: ['2025-02-20 10:59:11', 0.033, 0.847, 90.33, 32.4, 0.017, 0.437, 0.089]
📊 Data Saved: ['2025-02-20 10:59:12', 0.033, 0.847, 90.37, 32.43, 0.017, 0.437, 0.089]
📊 Data Saved: ['2025-02-20 10:59:13', 0.033, 0.847, 90.37, 32.43, 0.017, 0.437, 0.089]
📊 Data Saved: ['2025-02-20 10:59:14', 0.033, 0.847, 90.37, 32.43, 0.017, 0.437, 0.089]
📊 Data Saved: ['2025-02-20 10:59:16', 0.033, 0.847, 90.37, 32.43, 0.017, 0.437, 0.089]
📊 Data Saved: [

KeyboardInterrupt: 